In [64]:
from facenet_pytorch import InceptionResnetV1
import torch

# Load pretrained FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

In [65]:
from torchvision import transforms
from PIL import Image

# Define augmentation transformations
augmentations = transforms.Compose([
    transforms.RandomRotation(10),  
    transforms.RandomHorizontalFlip(),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(160, scale=(0.9, 1.0)),
    transforms.ToTensor()
])

def augment_image(image_path, num_augments=5):
    image = Image.open(image_path).convert('RGB')
    augmented_images = [augmentations(image) for _ in range(num_augments)]
    return augmented_images

In [ ]:
import torch.nn.functional as F

def get_embedding(image_path, model):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize((160, 160)), transforms.ToTensor()])
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        embedding = model(image)  # Get 128D face embedding
    return embedding.squeeze(0)  # Remove batch dimension

In [70]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np

detector = MTCNN()

def extract_face(image_path, required_size=(160, 160)):
    image = Image.open(image_path)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array



In [ ]:
import os

database = {}
image_folder = "hdataset/"  # Folder containing images of known people

for file in os.listdir(image_folder):
    if file.endswith(".jpeg") or file.endswith(".png"):
        person_name = file.split(".")[0]  # Assuming filename is "name.jpg"
        embedding = get_embedding(os.path.join(image_folder, file), model)
        database[person_name] = embedding

UnboundLocalError: cannot access local variable 'image' where it is not associated with a value

In [50]:
database['jaindu']

tensor([-5.9049e-03, -1.6267e-03, -7.5006e-02,  9.3720e-02, -5.8983e-03,
         9.7501e-02,  1.2672e-02, -1.3300e-02, -1.8464e-02, -8.9329e-02,
         1.1922e-02,  1.4288e-02,  7.4118e-02, -2.5966e-02, -3.5369e-02,
         3.9257e-02,  5.8211e-02,  4.4075e-02,  2.8584e-02, -6.6656e-02,
        -4.8641e-02, -6.7417e-02,  1.5841e-02, -1.2370e-02,  5.0445e-02,
        -6.4262e-03,  4.0469e-02, -3.5998e-02,  1.0906e-02,  5.9617e-02,
         4.0621e-02,  2.0166e-02, -2.4960e-02,  2.4997e-02, -4.8914e-02,
         3.9351e-02,  1.5049e-02, -2.7763e-02, -9.4885e-02,  9.6978e-02,
        -7.6364e-03, -3.9515e-02, -2.0563e-02, -2.0474e-02,  4.8121e-02,
        -2.7522e-03,  6.8501e-02,  1.0734e-01, -2.9871e-02, -1.4268e-02,
         7.9757e-03,  2.1395e-02,  4.4009e-02,  1.4203e-02, -3.8519e-02,
         2.1793e-02,  4.0790e-02,  3.1381e-02,  3.6213e-02,  4.9183e-03,
         5.7170e-02,  3.5531e-02,  2.6345e-02, -4.7825e-02, -3.3016e-02,
         9.8719e-02,  4.1848e-02, -2.6135e-03, -4.0

In [51]:
from scipy.spatial.distance import cosine

def recognize_face(test_image_path, model, database, threshold=0.5):
    test_embedding = get_embedding(test_image_path, model)

    min_distance = float('inf')
    best_match = None

    for person_name, stored_embedding in database.items():
        distance = cosine(test_embedding.numpy(), stored_embedding.numpy())  # Compute similarity
        if distance < min_distance:
            min_distance = distance
            best_match = person_name

    if min_distance < threshold:  # Set a threshold for recognition
        return best_match
    else:
        return "Unknown"

In [63]:
test_image = "test5.jpeg"
result = recognize_face(test_image, model, database)
print(f"Predicted identity: {result}")

Predicted identity: sasith
